In [1]:
import warnings as txt_sum_wrngs
txt_sum_wrngs.filterwarnings("ignore")

In [2]:
!pip install transformers
!pip install rouge-score
!pip install sentencepiece
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 954.5 kB/s eta 0:00:00a 0:00:01


In [3]:
import pandas as txt_sum_pd
import torch as txt_sum_trch
import sentencepiece
from transformers import T5Tokenizer as txt_sum_t5_tknzr
from transformers import T5ForConditionalGeneration as txt_sum_t5_cg
from transformers import pipeline as txt_sum_rg_ppln
from rouge_score import rouge_scorer as txt_sum_rg_scr

ModuleNotFoundError: No module named 'torch'

In [ ]:
txt_sum_data_frm = txt_sum_pd.read_csv("Data.csv")
txt_sum_data_frm.shape

In [ ]:
txt_sum_data_frm

In [ ]:
txt_sum_data_frm['Proceedings Content'].str.len()

#### Summarizing one sentence from the data using T5 as an example to understand the before and after results of summarization

In [ ]:
txt_sum_ex_text = txt_sum_data_frm["Proceedings Content"][0]
print(len(txt_sum_ex_text))

In [ ]:
def t5_txt_summarizer(txt_sum_inpt_txt):
  txt_sum_t5_mdl_nm = "t5-small"
  txt_sum_t5_mx_lth=200
  txt_sum_toknzr = txt_sum_t5_tknzr.from_pretrained(txt_sum_t5_mdl_nm)
  txt_sum_t5_mdl = txt_sum_t5_cg.from_pretrained(txt_sum_t5_mdl_nm)
  txt_sum_t5_inputs = txt_sum_toknzr.encode("summarize: " + txt_sum_inpt_txt, return_tensors="pt", max_length=1024, truncation=True)
  txt_sum_summary_ids = txt_sum_t5_mdl.generate(txt_sum_t5_inputs, max_length=txt_sum_t5_mx_lth, num_beams=4, early_stopping=True)
  txt_sum_output = txt_sum_toknzr.decode(txt_sum_summary_ids[0], skip_special_tokens=True)
  return txt_sum_output


In [ ]:
txt_sum_ex_output = t5_txt_summarizer(txt_sum_ex_text)

In [ ]:
print(len(txt_sum_ex_output))

In [ ]:
txt_sum_mdl_nm = "t5-small"
txt_sum_smrzr = txt_sum_rg_ppln("summarization", model=txt_sum_mdl_nm)

txt_sum_opt_smry = txt_sum_ex_output
txt_sum_inpt_ex = txt_sum_ex_text

txt_sum_scr = txt_sum_rg_scr.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
txt_sum_scrs = txt_sum_scr.score(txt_sum_inpt_ex, txt_sum_opt_smry)

for txt_sum_mtrc, txt_sum_scr in txt_sum_scrs.items():
    print(f"{txt_sum_mtrc}:")
    print(f"  Precision: {txt_sum_scr.precision:.4f}")
    print(f"  Recall: {txt_sum_scr.recall:.4f}")
    print(f"  F1 Score: {txt_sum_scr.fmeasure:.4f}")

#### Summarizing all the sentences in the data


In [ ]:
# defining the model to summarize the text data

def t5_txt_summarizer(txt_sum_inpt_txt):
  txt_sum_t5_mdl_nm = "t5-small"
  txt_sum_t5_mx_lth=200
  txt_sum_toknzr = txt_sum_t5_tknzr.from_pretrained(txt_sum_t5_mdl_nm)
  txt_sum_t5_mdl = txt_sum_t5_cg.from_pretrained(txt_sum_t5_mdl_nm)
  txt_sum_t5_inputs = txt_sum_toknzr.encode("summarize: " + txt_sum_inpt_txt, return_tensors="pt", max_length=1024, truncation=True)
  txt_sum_summary_ids = txt_sum_t5_mdl.generate(txt_sum_t5_inputs, max_length=txt_sum_t5_mx_lth, num_beams=4, early_stopping=True)
  txt_sum_output = txt_sum_toknzr.decode(txt_sum_summary_ids[0], skip_special_tokens=True)
  return txt_sum_output

In [ ]:
txt_sum_output_lis = []
for txt in txt_sum_data_frm['Proceedings Content']:
  txt_sum_smrzd_opt = t5_txt_summarizer(txt)
  txt_sum_output_lis.append(txt_sum_smrzd_opt)

In [ ]:
txt_sum_opt_dta_frm = txt_sum_pd.DataFrame()
for opt in range(len(txt_sum_output_lis)):
  txt_sum_pre = txt_sum_pd.Series({'Proceeding_content':txt_sum_data_frm['Proceedings Content'].unique()[opt],'T5_summarized_output':txt_sum_output_lis[opt]})
  txt_sum_opt_dta_frm = txt_sum_opt_dta_frm.append(txt_sum_pre,ignore_index=True)
txt_sum_opt_dta_frm['Category']=txt_sum_data_frm['Category']
txt_sum_opt_dta_frm

In [ ]:
txt_sum_opt_dta_frm['T5_summarized_output'].str.len()

Evaluation using Rougue Score

In [ ]:
txt_sum_mdl_nm = "t5-small"
txt_sum_smrzr = txt_sum_rg_ppln("summarization", model=txt_sum_mdl_nm)

row_index = 0

txt_sum_opt_smry = txt_sum_opt_dta_frm['T5_summarized_output'].iloc[row_index]
txt_sum_inpt_ex = txt_sum_opt_dta_frm["Proceeding_content"].iloc[row_index]

txt_sum_scr = txt_sum_rg_scr.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
txt_sum_scrs = txt_sum_scr.score(txt_sum_inpt_ex, txt_sum_opt_smry)

for txt_sum_mtrc, txt_sum_scr in txt_sum_scrs.items():
    print(f"{txt_sum_mtrc}:")
    print(f"  Precision: {txt_sum_scr.precision:.4f}")
    print(f"  Recall: {txt_sum_scr.recall:.4f}")
    print(f"  F1 Score: {txt_sum_scr.fmeasure:.4f}")